In [1]:
import os
import numpy as np

##### Loading the Original Data from PEMS07 Dataset

In [3]:
graph_signal_matrix_filename = r"C:\Users\User\Traffic_Flow_Forecasting\ASTGCN_Baseline_Model\PEMS04\PEMS04.npz"
data = np.load(graph_signal_matrix_filename)
data['data'].shape

(16992, 307, 3)

#### The above dataset PEMS07 has 883 traffic sensors recording data with a duration of 3 months 

In [7]:
number_of_days = 16992 / (12 * 24)
print("The required number of days for which the data is collected is ", number_of_days)

The required number of days for which the data is collected is  59.0


##### the 30 minutes data(INPUT) is of shape (12, 883, 1) and the next hour(TARGET) is the same (6, 883, 1)

#### The function search_data returns a certain number of time period segment data of duration = 1hr

In [9]:
def search_data(sequence_length, num_of_depend, label_start_idx,num_for_predict, units, points_per_quarterly_hour):
    '''
    Parameters
    ----------
    sequence_length: int, length of all history data
    num_of_depend: int, number of dependent time periods of length 1 hour
    label_start_idx: int, the first index of predicting target
    num_for_predict: int, the number of points will be predicted for each sample (in case of a hour, for each input sample 12 points to
                                                                                   be predicted)
    units: int, week: 7 * 24, day: 24, recent(hour): 1
    
    points_per_hour: int, number of points per hour, depends on data, here 12 points are considered because 1 hour is considered
    Returns
    ----------
    list[(start_idx, end_idx)]
    '''
    ###Checks if there are non-zero number of points in target sample
    if points_per_quarterly_hour < 0:
        raise ValueError("points_per_quarterly_hour should be greater than 0!")

    ###Checking if the selected sequence is bypassing the length of sequence
    if label_start_idx + num_for_predict > sequence_length:
        return None

    x_idx = []
    for i in range(1, num_of_depend + 1):
        start_idx = label_start_idx - points_per_quarterly_hour * units * i
        end_idx = start_idx + num_for_predict
        if start_idx >= 0:
            x_idx.append((start_idx, end_idx))
        else:
            return None
    ###Checking if we have same number of time periods or not
    if len(x_idx) != num_of_depend:
        return None
    ### return in from of recent time period segmnent order
    return x_idx[::-1]

### The Function 'get_sample_indices' returns the sample data concatenatedwith input (sequence length data), output (target) 

In [15]:
def get_sample_indices(data_sequence, num_of_weeks, num_of_days, num_of_quartelry_hours, label_start_idx, num_for_predict, points_per_quarterly_hour=3):
    '''
    Parameters
    ----------
    data_sequence: np.ndarray shape is (sequence_length, num_of_vertices, num_of_features)
    num_of_weeks, num_of_days, num_of_hours: int
    label_start_idx: int, the first index of predicting target
    num_for_predict: int, the number of points will be predicted for each sample
    points_per_half_hour: int, default 6, number of points in 30 mins
    Returns
    ----------
    week_sample: np.ndarray shape is (num_of_weeks * points_per_hour, num_of_vertices, num_of_features)
    day_sample: np.ndarray shape is (num_of_days * points_per_hour,  num_of_vertices, num_of_features)
    half_hour_sample: np.ndarray shape is (num_of_half_hour * points_per_half_hour, num_of_vertices, num_of_features)
    target: np.ndarray shape is (num_for_predict, num_of_vertices, num_of_features)
    '''
    week_sample, day_sample, quarterly_hour_sample = None, None, None

    if label_start_idx + num_for_predict > data_sequence.shape[0]:
        return None, None, None, None
   
    if num_of_quartelry_hours > 0:
        quarterly_hour_indices = search_data(data_sequence.shape[0], num_of_quartelry_hours, label_start_idx, num_for_predict, 1, points_per_quarterly_hour)
        if not quarterly_hour_indices:
            return None, None, None, None
        quarterly_hour_sample = np.concatenate([data_sequence[i: j] for i, j in quarterly_hour_indices], axis=0)
    

    if num_of_days > 0:
        day_indices = search_data(data_sequence.shape[0], num_of_days, label_start_idx, num_for_predict, 24, points_per_hour)
        if not day_indices:
            return None, None, None, None
        day_sample = np.concatenate([data_sequence[i: j] for i, j in day_indices], axis=0)

    if num_of_weeks > 0:
        week_indices = search_data(data_sequence.shape[0], num_of_weeks, label_start_idx, num_for_predict, 7 * 24, points_per_hour)
        if not week_indices:
            return None, None, None, None
        week_sample = np.concatenate([data_sequence[i: j] for i, j in week_indices], axis=0)

    target = data_sequence[label_start_idx: label_start_idx + num_for_predict]

    return week_sample, day_sample, quarterly_hour_sample, target

In [17]:
def read_and_generate_dataset(graph_signal_matrix_filename, num_of_weeks, num_of_days, num_of_quartelry_hours, num_for_predict, points_per_quarterly_hour=3):
    '''
    Parameters
    ----------
    graph_signal_matrix_filename: str, path of graph signal matrix file
    num_of_weeks, num_of_days, num_of_hours: int
    num_for_predict: int
    points_per_half_hour: int, default 6, depends on data
    Returns
    ----------
    feature: np.ndarray, shape is (num_of_samples, num_of_depend * points_per_hour, num_of_vertices, num_of_features)
    target: np.ndarray, shape is (num_of_samples, num_of_vertices, num_for_predict)
    '''
    #--------------------------------- Read original data 
    data_seq = np.load(graph_signal_matrix_filename)['data']  # (sequence_length, num_of_vertices, num_of_features) (16992, 307, 3)
    
    #---------------------------------
    all_samples = []
    for idx in range(data_seq.shape[0]):
        sample = get_sample_indices(data_seq, num_of_weeks, num_of_days, num_of_quartelry_hours, idx, num_for_predict, points_per_quarterly_hour)
        if ((sample[0] is None) and (sample[1] is None) and (sample[2] is None)):
            continue

        week_sample, day_sample, quarterly_hour_sample, target = sample #  week_sample, day_sample are None because we are predicting per hour
        #print(target.shape) # hour_sample and target (12, 307, 3)
        sample = []  # [(week_sample),(day_sample),(hour_sample),target,time_sample]
#-------------------------------- Ignore
        if num_of_weeks > 0:
            week_sample = np.expand_dims(week_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(week_sample)

        if num_of_days > 0:
            day_sample = np.expand_dims(day_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(day_sample)

        if num_of_quartelry_hours > 0:
            quarterly_hour_sample = np.expand_dims(quarterly_hour_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(quarterly_hour_sample)
        
        target = np.expand_dims(target, axis=0).transpose((0, 2, 3, 1))[:, :, 0, :]  # (1,N,T)
        sample.append(target)
        time_sample = np.expand_dims(np.array([idx]), axis=0)  # (1,1)
        sample.append(time_sample)
        all_samples.append(sample)#sampe：[(week_sample),(day_sample),(hour_sample),target,time_sample] = [(1,N,F,Tw),(1,N,F,Td),(1,N,F,Th),(1,N,Tpre),(1,1)]

    split_line1 = int(len(all_samples) * 0.6)
    split_line2 = int(len(all_samples) * 0.8)

    training_set = [np.concatenate(i, axis=0)  for i in zip(*all_samples[:split_line1])] #[(B,N,F,Tw),(B,N,F,Td),(B,N,F,Th),(B,N,Tpre),(B,1)]
    validation_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line1: split_line2])]
    testing_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line2:])]

    return training_set, validation_set, testing_set

In [19]:
num_of_vertices = 307
points_per_quarterly_hour = 3
num_for_predict = 3
num_of_weeks = 0
num_of_days = 0

###Such that the input sequence consists of
### data of consecutive 30 mins
num_of_quarterly_hours = 4

# Generating the training, test and validation dataset
training_set, validation_set, testing_set = read_and_generate_dataset(
    graph_signal_matrix_filename, 
    num_of_weeks, 
    num_of_days, 
    num_of_quarterly_hours, 
    num_for_predict, 
    points_per_quarterly_hour=points_per_quarterly_hour
)

In [21]:
len(training_set)##training set contains input, target and timestamp

3

In [23]:
training_set[0].shape

(10186, 307, 3, 12)

In [25]:
training_set[1].shape

(10186, 307, 3)

In [27]:
training_set[2].shape

(10186, 1)

In [29]:
training_set[0][0]

array([[[6.20e+01, 6.10e+01, 7.10e+01, ..., 1.64e+02, 1.43e+02,
         1.42e+02],
        [7.70e-03, 7.40e-03, 9.30e-03, ..., 2.15e-02, 1.91e-02,
         1.90e-02],
        [6.79e+01, 6.73e+01, 6.84e+01, ..., 6.57e+01, 6.59e+01,
         6.67e+01]],

       [[5.60e+01, 4.00e+01, 2.80e+01, ..., 1.51e+02, 1.07e+02,
         1.38e+02],
        [1.12e-02, 8.00e-03, 6.80e-03, ..., 3.40e-02, 2.69e-02,
         3.61e-02],
        [6.84e+01, 6.86e+01, 6.74e+01, ..., 6.40e+01, 6.33e+01,
         6.04e+01]],

       [[9.00e+01, 9.40e+01, 7.90e+01, ..., 1.58e+02, 1.39e+02,
         9.30e+01],
        [1.43e-02, 1.56e-02, 1.22e-02, ..., 2.69e-02, 2.34e-02,
         1.61e-02],
        [6.82e+01, 6.78e+01, 6.87e+01, ..., 6.71e+01, 6.67e+01,
         6.68e+01]],

       ...,

       [[5.60e+01, 5.10e+01, 6.70e+01, ..., 1.82e+02, 2.25e+02,
         1.72e+02],
        [9.80e-03, 9.40e-03, 1.17e-02, ..., 3.23e-02, 3.87e-02,
         2.96e-02],
        [6.74e+01, 6.60e+01, 6.67e+01, ..., 6.53e+01, 6.5

In [31]:
training_set[0][0].shape

(307, 3, 12)

In [33]:
training_set[1][0]

array([[145., 162., 126.],
       [141., 102., 127.],
       [132.,  96., 109.],
       [102.,  83.,  91.],
       [ 31.,  51.,  25.],
       [ 38.,  38.,  30.],
       [ 38.,  47.,  42.],
       [ 44.,  52.,  47.],
       [103.,  99.,  89.],
       [ 83.,  64.,  88.],
       [251., 235., 245.],
       [ 88.,  78.,  72.],
       [329., 256., 289.],
       [171., 194., 176.],
       [ 70.,  79.,  73.],
       [135., 149., 130.],
       [182., 211., 176.],
       [212., 194., 224.],
       [287., 213., 241.],
       [ 55., 187., 218.],
       [110.,  97.,  90.],
       [ 66.,  62.,  49.],
       [ 48.,  50.,  52.],
       [ 90.,  81.,  44.],
       [ 89., 131., 100.],
       [ 82.,  95.,  90.],
       [253., 229., 253.],
       [ 49.,  42.,  31.],
       [210., 259., 266.],
       [154., 137., 142.],
       [276., 327., 301.],
       [120., 125., 134.],
       [257., 201., 257.],
       [132., 120., 117.],
       [142., 145., 138.],
       [131., 136., 139.],
       [141., 110., 120.],
 

In [35]:
training_set[1][0].shape

(307, 3)

In [37]:
training_set[2][0]

array([12])

### Data Normalization

###### the data are transformed by zero-mean normalization x′ = x −mean(x) to let the average be 0

In [39]:
def normalization(train, val, test):
    '''
    Parameters
    ----------
    train, val, test: np.ndarray (B,N,F,T)
    Returns
    ----------
    stats: dict, two keys: mean and std
    train_norm, val_norm, test_norm: np.ndarray,
                                     shape is the same as original
    '''

    assert train.shape[1:] == val.shape[1:] and val.shape[1:] == test.shape[1:]  # ensure the num of nodes is the same
    mean = train.mean(axis=(0,1,3), keepdims=True)
    std = train.std(axis=(0,1,3), keepdims=True)
    print('mean.shape:',mean.shape)
    print('std.shape:',std.shape)

    def normalize(x):
        return (x - mean) / std

    train_norm = normalize(train)
    val_norm = normalize(val)
    test_norm = normalize(test)

    return {'_mean': mean, '_std': std}, train_norm, val_norm, test_norm

In [41]:
train_x = np.concatenate(training_set[:-2], axis=-1)  # (B,N,F,T')
val_x = np.concatenate(validation_set[:-2], axis=-1)
test_x = np.concatenate(testing_set[:-2], axis=-1)

train_target = training_set[-2]  # (B,N,T)
val_target = validation_set[-2]
test_target = testing_set[-2]

train_timestamp = training_set[-1]  # (B,1)
val_timestamp = validation_set[-1]
test_timestamp = testing_set[-1]

(stats, train_x_norm, val_x_norm, test_x_norm) = normalization(train_x, val_x, test_x)

all_data = {'train': { 'x': train_x_norm, 'target': train_target,'timestamp': train_timestamp},
            'val': {'x': val_x_norm, 'target': val_target, 'timestamp': val_timestamp},
            'test': {'x': test_x_norm, 'target': test_target, 'timestamp': test_timestamp},
            'stats': {'_mean': stats['_mean'], '_std': stats['_std']} }

mean.shape: (1, 1, 3, 1)
std.shape: (1, 1, 3, 1)


In [43]:
print('train x:', all_data['train']['x'].shape)
print('train target:', all_data['train']['target'].shape)
print('train timestamp:', all_data['train']['timestamp'].shape)
print()
print('val x:', all_data['val']['x'].shape)
print('val target:', all_data['val']['target'].shape)
print('val timestamp:', all_data['val']['timestamp'].shape)
print()
print('test x:', all_data['test']['x'].shape)
print('test target:', all_data['test']['target'].shape)
print('test timestamp:', all_data['test']['timestamp'].shape)
print()
print('train data _mean :', all_data['stats']['_mean'].shape, all_data['stats']['_mean'])
print('train data _std :', all_data['stats']['_std'].shape, all_data['stats']['_std'])

train x: (10186, 307, 3, 12)
train target: (10186, 307, 3)
train timestamp: (10186, 1)

val x: (3396, 307, 3, 12)
val target: (3396, 307, 3)
val timestamp: (3396, 1)

test x: (3396, 307, 3, 12)
test target: (3396, 307, 3)
test timestamp: (3396, 1)

train data _mean : (1, 1, 3, 1) [[[[2.07273288e+02]
   [5.13390651e-02]
   [6.34710210e+01]]]]
train data _std : (1, 1, 3, 1) [[[[1.56485751e+02]
   [4.78750672e-02]
   [8.10780143e+00]]]]


In [45]:
file = os.path.basename(graph_signal_matrix_filename).split('.')[0]
dirpath = '.'
filename = os.path.join(dirpath, file + '_304r' + str(num_of_quarterly_hours) + '_304d' + str(num_of_days) + '_304w' + str(num_of_weeks)) + '_astcgn'
print('save file:', filename)
np.savez_compressed(filename,
                train_x=all_data['train']['x'],train_target=all_data['train']['target'],train_timestamp=all_data['train']['timestamp'],
                val_x=all_data['val']['x'], val_target=all_data['val']['target'],val_timestamp=all_data['val']['timestamp'],
                test_x=all_data['test']['x'], test_target=all_data['test']['target'], test_timestamp=all_data['test']['timestamp'],
                mean=all_data['stats']['_mean'], std=all_data['stats']['_std'])

save file: .\PEMS04_304r4_304d0_304w0_astcgn
